In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
# determining the learnable parameters
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
print(params[1].size())  # conv2's .weight
print(params[2].size())  # fc1's .weight
print(params[3].size())  # 
print(params[4].size())  # 
print(params[5].size())  # 
print(params[6].size())  # fc2's .weight
print(params[7].size())  # fc2's bias ?
print(params[8].size())  # fc3's .weight
print(params[9].size())  # fc3'a bias

10
torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([16, 6, 3, 3])
torch.Size([16])
torch.Size([120, 576])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [4]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(input)
print(out)

tensor([[[[-2.1250, -0.1200, -0.0141,  ...,  0.7247,  1.3915, -0.9906],
          [-0.3084,  0.0749, -1.2372,  ..., -0.4601,  0.7420, -2.3440],
          [-0.8126,  0.9832,  2.0887,  ..., -0.0638, -0.1636,  0.1702],
          ...,
          [-0.5132, -0.9929, -0.5064,  ...,  0.1062, -0.3050,  0.8276],
          [-2.1982, -0.6384,  0.0754,  ...,  1.3483,  1.1394, -0.7303],
          [-0.8975,  0.1721,  1.4573,  ...,  0.2046, -2.0546, -0.2635]]]])
tensor([[-0.1012, -0.0893,  0.0107,  0.0274, -0.0996, -0.0266,  0.0456,  0.0273,
         -0.0779,  0.0923]], grad_fn=<AddmmBackward>)


In [5]:
net.zero_grad() # zero the gradient buffers
out.backward(torch.randn(1, 10))

In [6]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
print(output)
print(target)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor([[-0.1012, -0.0893,  0.0107,  0.0274, -0.0996, -0.0266,  0.0456,  0.0273,
         -0.0779,  0.0923]], grad_fn=<AddmmBackward>)
tensor([[-0.4794,  0.9709, -0.4930, -0.2126,  1.7812, -1.0723,  0.9750, -0.9314,
         -0.3945, -0.7018]])
tensor(0.8723, grad_fn=<MseLossBackward>)


In [7]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [8]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0099,  0.0089,  0.0075,  0.0083, -0.0103, -0.0059])


In [9]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [10]:
# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

In [11]:
print(output)

tensor([[-0.1026, -0.0829,  0.0090,  0.0268, -0.0915, -0.0307,  0.0512,  0.0228,
         -0.0814,  0.0892]], grad_fn=<AddmmBackward>)
